In [2]:
import $file.QaSpark
import QaSpark._

import $file.$      

import QaSpark._

## NY Taxi

In [3]:
var nyt = spark
    .read
    .option("inferSchema", "true")
    .option("header", "true")
    .csv("ny-taxi/test.csv")

nyt: org.apache.spark.sql.package.DataFrame = [ID: string, Vendor: int ... 7 more fields]

In [4]:
nyt.count()

res3: Long = 702L

In [5]:
nyt.columns

res4: Array[String] = Array(
  "id",
  "vendor_id",
  "pickup_datetime",
  "passenger_count",
  "pickup_longitude",
  "pickup_latitude",
  "dropoff_longitude",
  "dropoff_latitude",
  "store_and_fwd_flag"
)

In [6]:
nyt.schema

res5: org.apache.spark.sql.types.StructType = StructType(
  StructField("id", StringType, true, {}),
  StructField("vendor_id", IntegerType, true, {}),
  StructField("pickup_datetime", StringType, true, {}),
  StructField("passenger_count", IntegerType, true, {}),
  StructField("pickup_longitude", DoubleType, true, {}),
  StructField("pickup_latitude", DoubleType, true, {}),
  StructField("dropoff_longitude", DoubleType, true, {}),
  StructField("dropoff_latitude", DoubleType, true, {}),
  StructField("store_and_fwd_flag", StringType, true, {})
)

For shorter demonstration output, trimming column names:

In [7]:
nyt = nyt.selectExpr(
    "id as ID",  
    "vendor_id as Vendor",
    "pickup_datetime as UpTime",
    "passenger_count as Passengers",
    "pickup_longitude as UpLong",
    "pickup_latitude as UpLat",
    "dropoff_longitude as DropLong",
    "dropoff_latitude as DropLat",
    "store_and_fwd_flag as IsLive"
)

## Type Safety with Datasets

In [8]:
Qa.setScope(this)

import spark.implicits._

case class Trip(
    ID : String, 
    Vendor : Int ,
    UpTime : String, 
    Passengers : Int, 
    UpLong : Double, 
    UpLat : Double, 
    DropLong : Double, 
    DropLat : Double, 
    IsLive : String
)





val ds = nyt.as[Trip]

import spark.implicits._


defined class Trip
ds: org.apache.spark.sql.Dataset[Trip] = [ID: string, Vendor: int ... 7 more fields]

In [9]:
nyt.take(5)

res8: Array[org.apache.spark.sql.Row] = Array(
  [id3004672,1,30/06/2016 23:59,1,-73.98812866,40.73202896,-73.99017334,40.75667953,N],
  [id3505355,1,30/06/2016 23:59,1,-73.96420288,40.67999268,-73.95980835,40.65540314,N],
  [id1217141,1,30/06/2016 23:59,1,-73.99743652,40.73758316,-73.98616028,40.72952271,N],
  [id2150126,2,30/06/2016 23:59,1,-73.95606995,40.77190018,-73.98642731,40.73046875,N],
  [id1598245,1,30/06/2016 23:59,1,-73.97021484,40.76147461,-73.9615097,40.75588989,N]
)

In [10]:
ds.take(2)

res9: Array[Trip] = Array(
  Trip(
    "id3004672",
    1,
    "30/06/2016 23:59",
    1,
    -73.98812866,
    40.73202896,
    -73.99017334,
    40.75667953,
    "N"
  ),
  Trip(
    "id3505355",
    1,
    "30/06/2016 23:59",
    1,
    -73.96420288,
    40.67999268,
    -73.95980835,
    40.65540314,
    "N"
  )
)

In [11]:
nyt

res10: org.apache.spark.sql.package.DataFrame = [ID: string, Vendor: int ... 7 more fields]

In [12]:
Qa.show(nyt)

id3004672,1,30/06/2016,1,-73.988128,40.7320289,-73.990173,40.7566795,N
id3505355,1,30/06/2016,1,-73.964202,40.6799926,-73.959808,40.6554031,N
id1217141,1,30/06/2016,1,-73.997436,40.7375831,-73.986160,40.7295227,N
id2150126,2,30/06/2016,1,-73.956069,40.7719001,-73.986427,40.7304687,N
id1598245,1,30/06/2016,1,-73.970214,40.7614746,-73.961509,40.7558898,N


In [13]:
nyt.show(3, 10)

+---------+------+----------+----------+----------+----------+----------+----------+------+
|       ID|Vendor|    UpTime|Passengers|    UpLong|     UpLat|  DropLong|   DropLat|IsLive|
+---------+------+----------+----------+----------+----------+----------+----------+------+
|id3004672|     1|30/06/2...|         1|-73.988...|40.7320...|-73.990...|40.7566...|     N|
|id3505355|     1|30/06/2...|         1|-73.964...|40.6799...|-73.959...|40.6554...|     N|
|id1217141|     1|30/06/2...|         1|-73.997...|40.7375...|-73.986...|40.7295...|     N|
+---------+------+----------+----------+----------+----------+----------+----------+------+
only showing top 3 rows



In [14]:
ds
 .filter(row => row.Passengers > 1)
 .map(r => r.Vendor)
 .show(5)

+-----+
|value|
+-----+
|    1|
|    2|
|    1|
|    2|
|    2|
+-----+
only showing top 5 rows



In [15]:
Qa.setScope(this)

case class TripSummary(passengers: Double, time: String)

ds
 .filter(row => row.Passengers > 1)
 .map(r => TripSummary(r.Passengers, r.UpTime)) 
 .show(5)

+----------+----------------+
|passengers|            time|
+----------+----------------+
|       2.0|30/06/2016 23:59|
|       2.0|30/06/2016 23:58|
|       4.0|30/06/2016 23:58|
|       5.0|30/06/2016 23:55|
|       5.0|23/01/2016 21:15|
+----------+----------------+
only showing top 5 rows



defined class TripSummary

## Structured Streaming

* createOrReplaceTempView -- a view to run sql over

In [16]:
nyt.createOrReplaceTempView("taxi")

In [17]:
import org.apache.spark.sql.functions.{window, column, desc, col}

var nytDays = spark
  .read
  .schema(nyt.schema)
  .option("header", "false")
  .csv("ny-taxi/2016.*.csv")

nytDays
  .select("Passengers","UpTime")
  .groupBy(window(col("UpTime"), "1 day"))
  .sum("Passengers")
  .show(5)


+--------------------+---------------+
|              window|sum(Passengers)|
+--------------------+---------------+
|[2016-03-18 00:00...|           6506|
|[2016-06-04 00:00...|           6292|
|[2016-05-28 00:00...|           5183|
|[2016-05-22 00:00...|           6031|
|[2016-02-24 00:00...|           5886|
+--------------------+---------------+
only showing top 5 rows



import org.apache.spark.sql.functions.{window, column, desc, col}


nytDays: org.apache.spark.sql.package.DataFrame = [ID: string, Vendor: int ... 7 more fields]

In [18]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

val nytStream = spark.readStream
 .schema(nyt.schema)
 .option("maxFilesPerTrigger", 1)
 .option("header", "false")
 .csv("ny-taxi/2016.*.csv")

nytStream: org.apache.spark.sql.package.DataFrame = [ID: string, Vendor: int ... 7 more fields]

In [19]:
nytStream.isStreaming

res18: Boolean = true

In [20]:
val passengersPerDay = nytStream
  .select("Passengers","UpTime")
  .groupBy(window($"UpTime", "1 day"))
  .sum("Passengers")

passengersPerDay: org.apache.spark.sql.package.DataFrame = [window: struct<start: timestamp, end: timestamp>, sum(Passengers): bigint]

In [21]:
passengersPerDay.writeStream
 .format("memory")
 .queryName("per_day")
 .outputMode("complete")
 .start()

res20: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@270baabc

In [33]:
spark.sql("SELECT * FROM per_day").show(5)

+--------------------+---------------+
|              window|sum(Passengers)|
+--------------------+---------------+
|[2016-06-19 00:00...|           4965|
|[2016-05-31 00:00...|           5200|
|[2016-04-03 00:00...|           6253|
|[2016-04-27 00:00...|           5346|
|[2016-06-21 00:00...|           5573|
+--------------------+---------------+
only showing top 5 rows



## Machine Learning

In [23]:
import org.apache.spark.sql.functions.date_format

val df = nytDays
  .na.fill(0)
  .withColumn("Weekday", date_format($"UpTime", "EEEE"))
  .coalesce(5)

val xTrain = df.where("UpTime <= '2016-01-31'")
val xTest = df.where("UpTime > '2016-01-31'")

import org.apache.spark.sql.functions.date_format


df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ID: string, Vendor: int ... 8 more fields]
xTrain: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ID: string, Vendor: int ... 8 more fields]
xTest: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [ID: string, Vendor: int ... 8 more fields]

In [24]:
Qa.show(df)

id3660645,1,2016-04-09,1,-73.995094,40.7437744,-73.973571,40.7379722,N,Saturday
id2378926,2,2016-04-09,1,-73.974571,40.7565612,-73.925743,40.7642593,N,Saturday
id3198822,1,2016-04-09,3,-73.993690,40.7361259,-74.014625,40.7097930,N,Saturday
id2065219,2,2016-04-09,1,-73.974853,40.7566757,-73.990585,40.7286567,N,Saturday
id0805827,2,2016-04-09,1,-73.971664,40.7503852,-73.988059,40.7187461,N,Saturday


In [25]:
val count = (xTrain.count(), xTest.count())

println(count)
println(100 * count._1 / count._2)

(94441,530693)
17


count: (Long, Long) = (94441L, 530693L)

In [26]:
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.OneHotEncoder
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.Pipeline

val features = Array("Passengers", "UpLong", "UpLat",  "eWeekday")

val pipline = new Pipeline().setStages(Array(
     new StringIndexer()
        .setInputCol("Weekday")
        .setOutputCol("iWeekday")
    , new OneHotEncoder()
        .setInputCol("iWeekday")
        .setOutputCol("eWeekday")
    , new VectorAssembler()
        .setInputCols(features)
        .setOutputCol("features")
)).fit(xTrain)

import org.apache.spark.ml.feature.StringIndexer

import org.apache.spark.ml.feature.OneHotEncoder

import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.Pipeline


features: Array[String] = Array("Passengers", "UpLong", "UpLat", "eWeekday")
pipline: org.apache.spark.ml.PipelineModel = pipeline_cc5908a78dd2

In [27]:
val xTrainFeatures = pipline.transform(xTrain)
val xTestFeatures = pipline.transform(xTest)

xTrainFeatures.cache()

xTrainFeatures: org.apache.spark.sql.package.DataFrame = [ID: string, Vendor: int ... 11 more fields]
xTestFeatures: org.apache.spark.sql.package.DataFrame = [ID: string, Vendor: int ... 11 more fields]
res26_2: org.apache.spark.sql.package.DataFrame = [ID: string, Vendor: int ... 11 more fields]

In [28]:
import org.apache.spark.ml.clustering.KMeans

val clusters = new KMeans()
    .setK(20)
    .setSeed(1L)
    .fit(xTrainFeatures)

19/07/25 13:08:26 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
19/07/25 13:08:26 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


import org.apache.spark.ml.clustering.KMeans


clusters: org.apache.spark.ml.clustering.KMeansModel = kmeans_98f2759cf2ce

In [29]:
clusters.computeCost(xTrainFeatures)
clusters.computeCost(xTestFeatures)

res28_0: Double = 9283.067655640947
res28_1: Double = 55202.508320186214

In [32]:
Qa.show(features, clusters.clusterCenters.map( _.toArray.toSeq ))

1.0,-73.972278,40.7524288,0.0,0.0,0.0,0.0,1.0,0.0
4.70301418,-73.975730,40.7482898,0.0,1.0,0.0,0.0,0.0,0.0
1.99999999,-73.972213,40.7501122,0.0,0.0,0.0,0.99999999,0.0,0.0
4.79712683,-73.973668,40.7511515,0.45409950,0.0,0.0,0.24421864,0.0,0.0
0.99979879,-73.972923,40.7521134,0.0,0.0,0.0,1.0,0.0,0.0
2.21179039,-73.976989,40.7489507,0.0,1.0,0.0,0.0,0.0,0.0
1.0,-73.973732,40.7506507,1.0,0.0,0.0,0.0,0.0,0.0
1.0,-73.972615,40.7491589,0.0,0.0,0.0,0.0,0.0,0.0
2.21343873,-73.970790,40.7516268,0.0,0.0,1.0,0.0,0.0,0.0
3.16981132,-73.970857,40.7487218,0.0,0.0,0.0,0.0,0.0,0.55145797
1.0,-73.975641,40.7492162,0.0,1.0,0.0,0.0,0.0,0.0
